In [50]:
import os
import numpy as np
import git
import mlflow
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [51]:
# --- 1. Метрики и Scoring Functions ---

def calculate_metrics(y_true, y_pred):
    """
    Вычисляет расширенный набор метрик для RUL.
    y_true, y_pred: torch.Tensor или numpy array
    """
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.detach().cpu().numpy()
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.detach().cpu().numpy()
        
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    # Разница
    d = y_pred - y_true
    
    # 1. MAE
    mae = np.mean(np.abs(d))
    
    # 2. RMSE
    rmse = np.sqrt(np.mean(d**2))
    
    # 3. MAPE
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1.0))) * 100

    # 4. PHM08 Score (NASA Scoring Function) [web:PHM08_Challenge]
    # Функция асимметрична: ранние предсказания (d < 0) штрафуются меньше, чем поздние (d > 0)
    # Формула: sum(exp(-d/13) - 1 если d < 0, иначе exp(d/10) - 1)
    # *Внимание: в условии было указано score = sum(...) / n. Обычно в PHM08 используют просто sum,
    # но для сопоставимости метрик лучше использовать среднее (mean) или следовать условию задачи.
    # Здесь реализуем согласно вашему ТЗ: делим на n.
    n = len(d)
    scores = np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)
    phm08_score = np.sum(scores) / n
    
    return {"mae": mae, "rmse": rmse, "mape": mape, "phm08_score": phm08_score}

In [52]:
# --- 2. Адаптивная модель (Transfer Learning Ready) ---

class AdaptiveLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim_1=64, hidden_dim_2=32, output_dim=1, dropout_prob=0.2):
        super(AdaptiveLSTMModel, self).__init__()
        
        # === АДАПТИВНЫЙ ВХОДНОЙ СЛОЙ (ADAPTER) ===
        # Проецируем входные признаки (любого кол-ва) в фиксированное скрытое пространство (hidden_dim_1).
        # При Transfer Learning мы заменим только этот слой.
        self.input_adapter = nn.Linear(input_dim, hidden_dim_1)
        
        # === BACKBONE (ЯДРО МОДЕЛИ) ===
        # LSTM слои теперь принимают hidden_dim_1, а не input_dim.
        # Это позволяет весам LSTM оставаться валидными даже если input_dim изменится.
        self.backbone_lstm1 = nn.LSTM(hidden_dim_1, hidden_dim_1, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.backbone_lstm2 = nn.LSTM(hidden_dim_1, hidden_dim_2, batch_first=True)
        self.dropout2 = nn.Dropout(dropout_prob)
        
        # Финальная "голова"
        self.fc = nn.Linear(hidden_dim_2, output_dim)

    def forward(self, x):
        # x: (batch_size, seq_len, input_dim)
        
        # 1. Адаптация входа
        x_embedded = self.input_adapter(x) # -> (batch_size, seq_len, hidden_dim_1)
        
        # 2. Проход через Backbone
        lstm1_out, _ = self.backbone_lstm1(x_embedded)
        out = self.dropout1(lstm1_out)
        
        lstm2_out, _ = self.backbone_lstm2(out)
        
        # Берем выход последнего временного шага
        last_hidden_state = lstm2_out[:, -1, :] 
        out = self.dropout2(last_hidden_state)
        
        # 3. Финальный прогноз
        final_output = self.fc(out)
        return final_output

In [53]:
def adapt_for_customer_data(backbone_path, new_input_dim, hidden_dim_1=64, hidden_dim_2=32, dropout_prob=0.2):
    """
    Создает модель для новых данных заказчика, загружая предобученный backbone.
    """
    # 1. Инициализируем модель с НОВОЙ размерностью входа
    new_model = AdaptiveLSTMModel(
        input_dim=new_input_dim, # Например, 15 сенсоров вместо 20
        hidden_dim_1=hidden_dim_1,
        hidden_dim_2=hidden_dim_2,
        dropout_prob=dropout_prob
    )
    
    # 2. Загружаем веса Backbone
    # strict=False позволяет игнорировать несовпадение ключей (т.к. input_adapter у нас новый)
    pretrained_dict = torch.load(backbone_path)
    model_dict = new_model.state_dict()
    
    # Фильтруем, чтобы загрузить только backbone
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and "input_adapter" not in k}
    
    # Обновляем веса
    model_dict.update(pretrained_dict)
    new_model.load_state_dict(model_dict)
    
    # 3. Замораживаем Backbone (опционально, если хотим учить только адаптер)
    for name, param in new_model.named_parameters():
        if "input_adapter" not in name:
            param.requires_grad = False
            
    print(f"Model adapted for input_dim={new_input_dim}. Backbone weights loaded and frozen.")
    return new_model

In [ ]:
# Указываем MLflow, куда отправлять данные
mlflow.set_tracking_uri("http://213.21.252.250:5000")

# Задаем имя эксперимента
mlflow.set_experiment("LSTM_TransferLearning_Ready")

# --- Получаем хеш коммита Git ---
try:
    repo = git.Repo(search_parent_directories=True)
    git_commit_hash = repo.head.object.hexsha
except Exception as e:
    git_commit_hash = "N/A" # На случай, если скрипт запущен не из Git-репозитория
    print(f"Warning: Could not get git commit hash. {e}")

print(f"Current Git Commit Hash: {git_commit_hash}")

# --- Параметры, которые нужно логировать ---
# Параметры из скрипта нарезки данных (sample_creator)
data_params = {
    "window_size": 50,
    "step": 1,
    "sampling_rate": 5
}

# Гиперпараметры модели
model_params = {
    "epochs": 7,
    "batch_size": 128,
    "validation_split": 0.2,
    "optimizer": "adam",
    "loss": "mean_squared_error",
    "lr": 0.002,
    "hidden_dim_1": 32,
    "hidden_dim_2": 16,
    "dropout": 0.2
}

Current Git Commit Hash: 910fc0116571f29bf7c5df3b283d99b529fee69e


In [ ]:
def load_and_merge_data(npz_units):
      sample_array_lst = []
      label_array_lst = []
      for npz_unit in npz_units:
        loaded = np.load(npz_unit)
        sample_array_lst.append(loaded['sample'])
        label_array_lst.append(loaded['label'])
      sample_array = np.dstack(sample_array_lst)
      label_array = np.concatenate(label_array_lst)
      sample_array = sample_array.transpose(2, 0, 1)
      return sample_array, label_array

processed_dir = '../data/processed/'

# Собираем пути к файлам для train и test
train_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit2_', 'Unit5_', 'Unit10_', 'Unit16_', 'Unit18_', 'Unit20_'))]
test_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit11_', 'Unit14_', 'Unit15_'))]
print(train_files)

# Загружаем данные
X_train, y_train = load_and_merge_data(train_files)
X_test, y_test = load_and_merge_data(test_files)

# Определяем форму входных данных из X_train
n_timesteps, n_features = X_train.shape[1], X_train.shape[2]

print('Размер обучающей выборки (X):', X_train.shape)
print('Размер обучающей выборки (y):', y_train.shape)
print('Размер тестовой выборки (X):', X_test.shape)
print('Размер тестовой выборки (y):', y_test.shape)

['../data/processed/Unit20_win50_str1_smp1.npz', '../data/processed/Unit18_win50_str1_smp1.npz', '../data/processed/Unit5_win50_str1_smp1.npz', '../data/processed/Unit16_win50_str1_smp1.npz', '../data/processed/Unit2_win50_str1_smp1.npz', '../data/processed/Unit10_win50_str1_smp1.npz']


In [ ]:
with mlflow.start_run():
    print("Starting MLflow run...")

    # --- Логируем параметры ---
    mlflow.log_params(data_params)
    mlflow.log_params(model_params)
    mlflow.set_tag("git_commit", git_commit_hash)
    print("Parameters logged.")

    # --- Подготовка данных для PyTorch ---
    # 1. Преобразуем numpy массивы в torch тензоры
    X_train_tensor = torch.from_numpy(X_train).float()
    y_train_tensor = torch.from_numpy(y_train).float().view(-1, 1) # Убедимся, что форма (batch_size, 1)
    X_test_tensor = torch.from_numpy(X_test).float()
    y_test_tensor = torch.from_numpy(y_test).float().view(-1, 1)

    # 2. Создаем датасеты
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # 3. Разделяем на обучающую и валидационную выборки вручную
    val_split = model_params['validation_split']
    dataset_size = len(train_dataset)
    val_size = int(val_split * dataset_size)
    train_size = dataset_size - val_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

    # 4. Создаем загрузчики данных (DataLoader), которые будут подавать данные батчами
    train_loader = DataLoader(dataset=train_subset, batch_size=model_params['batch_size'], shuffle=True)
    val_loader = DataLoader(dataset=val_subset, batch_size=model_params['batch_size'])
    test_loader = DataLoader(dataset=test_dataset, batch_size=model_params['batch_size'])

    # Инициализация модели
    device = torch.device("cpu") # Требование: работать на CPU
    
    model = AdaptiveLSTMModel(
        input_dim=n_features,
        hidden_dim_1=model_params["hidden_dim_1"],
        hidden_dim_2=model_params["hidden_dim_2"],
        dropout_prob=model_params["dropout"]
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=model_params["lr"])

    # Цикл обучения
    for epoch in range(model_params["epochs"]):
        model.train()
        train_losses = []
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        avg_train_loss = np.mean(train_losses)
        mlflow.log_metric("train_mse_loss", avg_train_loss, step=epoch)

        # Валидация
        model.eval()
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_preds.append(outputs.numpy())
                val_targets.append(labels.numpy())
        
        val_preds = np.concatenate(val_preds)
        val_targets = np.concatenate(val_targets)
        
        # Расчет всех метрик
        val_metrics = calculate_metrics(val_targets, val_preds)
        
        # Логирование метрик валидации
        for name, value in val_metrics.items():
            mlflow.log_metric(f"val_{name}", value, step=epoch)
            
        print(f"Epoch {epoch+1}/{model_params['epochs']} | Train Loss: {avg_train_loss:.2f} | Val MAE: {val_metrics['mae']:.2f} | PHM08: {val_metrics['phm08_score']:.2f}")

    # --- 5. Финальный тест и сохранение ---
    print("\nEvaluating on Test Set...")
    model.eval()
    test_preds = []
    test_targets = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            test_preds.append(outputs.numpy())
            test_targets.append(labels.numpy())
    
    test_preds = np.concatenate(test_preds)
    test_targets = np.concatenate(test_targets)
    
    test_metrics = calculate_metrics(test_targets, test_preds)
    print(f"Test Metrics: {test_metrics}")
    
    # Логируем финальные метрики с префиксом test_
    for name, value in test_metrics.items():
        mlflow.log_metric(f"test_{name}", value)

    # 1. Сохранение ПОЛНОЙ модели
    mlflow.pytorch.log_model(model, "full_model")
    
    # 2. Сохранение BACKBONE (State Dict без входного слоя) для Transfer Learning
    # Исключаем веса input_adapter, чтобы клиент мог инициализировать свои
    backbone_state_dict = {k: v for k, v in model.state_dict().items() if "input_adapter" not in k}
    torch.save(backbone_state_dict, "lstm_backbone.pth")
    mlflow.log_artifact("lstm_backbone.pth", artifact_path="transfer_learning_artifacts")
    
    print("Run Complete. Artifacts logged.")

Starting MLflow run...
Parameters logged.
Epoch 1/5 | Train Loss: 663.87 | Val MAE: 19.18 | PHM08: 8.11
Epoch 2/5 | Train Loss: 518.21 | Val MAE: 19.18 | PHM08: 8.13
Epoch 3/5 | Train Loss: 516.46 | Val MAE: 19.18 | PHM08: 8.10
Epoch 4/5 | Train Loss: 151.47 | Val MAE: 5.45 | PHM08: 0.88
Epoch 5/5 | Train Loss: 75.99 | Val MAE: 4.54 | PHM08: 0.63

Evaluating on Test Set...
Test Metrics: {'mae': 4.3306284, 'rmse': 6.0068874, 'mape': 15.847097337245941, 'phm08_score': 0.6585267404393621}


2025/12/21 12:49:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/21 12:49:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run Complete. Artifacts logged.
🏃 View run clean-sow-59 at: http://213.21.252.250:5000/#/experiments/2/runs/9f000f382a8640cab5a8b98bc708448d
🧪 View experiment at: http://213.21.252.250:5000/#/experiments/2
